## live recision filter 
### for Hessas mest facebook paeg

> this code extract all revision videos from ***Hessas Mesr*** facebook posts data

### the project workflow is 

1- collects all posts ID/link <br />
2- get every post with title/attatchments <br />
3- select only video posts & filter revision videos from all the posts we have<br />
4- seperate the revision video from the orignal data <br />

#### Import libraries

In [1]:
import pandas as pd

from pyfacebook import GraphAPI as gpi
import requests
import json
import time
from datetime import datetime

#### init facebook API

In [2]:
# access  token info
access = {'access_token': '',
          'category': 'Education website',
          'category_list': [{'id': '2704', 'name': 'Education website'},
                            {'id': '2250', 'name': 'Education'}],
          'name': 'حصص مصر - Hesas Misr',
          'id': '',
          'tasks': ['ANALYZE', 'ADVERTISE', 'MESSAGING', 'MODERATE']}


# init facebook api
api = gpi(access_token=access['access_token'])


#### prepare the data
> read the page comments dataset <br />extract ***page id*** from ***comment id*** <br />add ***page_id*** columne and extract the posts

the comment id : { *"post_id"  __ "comment_id"* } <br> 
the post id : {*"page_id"  __  "post_id"* }


In [3]:
# readthe table
df = pd.read_csv(
    "C:\\Users\\20812018100700\\working\work\\Hessas_analysis\\full_DF.csv")
df.head()

,created_time,message,id,from,Translate,is_human,sentment,positive,neutral,negative
0,2021-09-06T04:18:01+0000,Mohammed Adel,271594718130347_271821218107697,NaN,Mohammed Adel,T,neutral,0.00,1.00,0.00
1,2021-09-05T19:13:35+0000,مش بيجي منها حاجة ارحمونا بقى,271594718130347_271610824795403,NaN,Nothing comes of it. Have mercy on us,F,negative,0.21,0.24,0.55
2,2021-09-06T05:06:15+0000,الفيزياء,271594718130347_271840278105791,NaN,Physics,F,neutral,0.13,0.83,0.04
3,2021-09-06T00:02:03+0000,نرجو الرافة بالطلاب في الدور الثاني ومراعاتهم ...,271594718130347_271721658117653,NaN,نرجو الرافة بالطلاب في الدور الثاني ومراعاتهم ...,T,neutral,0.00,1.00,0.00
4,2021-09-05T19:06:33+0000,نهايه رحله,271594718130347_271608141462338,NaN,the end of the trip,F,neutral,0.10,0.86,0.04


## 1- collects all posts ID/link

In [4]:
##split id on "_"
sep_id = df.id.str.split("_")

## extract posts id from seprated ids
posts_id = [ "{}_{}".format(access["id"], sep_id[i][0 ]) for i in range(len(sep_id))]

## create post_id columne
df["post_id"] = pd.Series(posts_id)
df[["id", "post_id"]].sample(5)

,id,post_id
42491,176431454313341_176522830970870,102899871666500_176431454313341
39712,978949249576165_978976059573484,102899871666500_978949249576165
44364,175007347580562_175134407567856,102899871666500_175007347580562
41910,184387556851064_187711229852030,102899871666500_184387556851064
6853,177079484460529_430555151338697,102899871666500_177079484460529


#### extract posts id
> we have 307 post in the page

In [5]:
# extract the posts id
ids_list = list(df.post_id.unique())
len(ids_list)

307

## 2- get every post with title/attatchments
> send request for all posts to get attatchments <br> send bulk ids per request 

In [6]:
# get posts attatchment info
# send request for5 0 posts at the same time
# joining 50 id and sends request

video_result = []
for i in range( len(ids_list) % 50 ):
    
    video_result.append(api.get_objects( ids=", ".join(ids_list[i*50:(i+1)*50]) ## join 50 id in one str with "," sep
                                        , fields="attachments"))

    print("ids from %d to %d" % (i*50, (i+1)*50))

ids from 0 to 50
ids from 50 to 100
ids from 100 to 150
ids from 150 to 200
ids from 200 to 250
ids from 250 to 300
ids from 300 to 350


In [7]:
##sellect all result in one array

video_posts_res = []
for i in video_result:
    video_posts_res.extend(i.values())


## 3- select only video posts & filter revision videos from all the posts we have
> init revision videos genere (extracted manually)

In [8]:
## all revision video genere
subjects = ["التفاضل والتكامل", "الجيولوجيا","الاستاتيكا","الأحياء","الديناميكا","علم النفس","اللغة الإنجليزية","التاريخ","الفيزياء","الكيمياء","الجغرافيا"
            ,"الجبر والهندسة الفراغية","للغة العربية","الألمانية","الإسبانية","الإيطالية","الفلسفة","لدراسات الاجتماعية","الرياضيات","الهندسة","العلوم","للغة الفرنسية",]

extract revision videos
> 1- loop for every post attatchment we have <br />
> 2- check if post have attatchment (video/image)<br />
> 3- check if attatchment type is video and have a title<br />
> 4- check if the video have any revision keywords<br />
> 5- chek if any subject name in the revision video title<br />
> 6- add subject name and post id

In [9]:
## check if object have attatchment with type is video
## check if the video have title [not a promot]
## check if keywords in the title 
## collect the post ID/subject 

video_posts_gen = []
video_posts_id = []
n = -1
for i in video_posts_res:
    n+=1
    if "attachments" in i.keys():
        
        k = i["attachments"]["data"][0]
        
        if (k['type'] == "video_inline") and ("title" in k.keys()):
            if("مادة" in k["title"] or "مراجعة" in k["title"] or "المراجعة" in k["title"]):
                print(n)   
                print(k["title"]) 
                for subject in subjects:
                    if subject in k["title"]:
                        video_posts_id.append(i["id"])
                        video_posts_gen.append(subject)
                        print(subject)
                        print("*"*50)
                        break

14
مراجعة ليلة الامتحان - التفاضل والتكامل
التفاضل والتكامل
**************************************************
15
مراجعة ليلة الامتحان - مادة الجيولوجيا
الجيولوجيا
**************************************************
18
مراجعة مادة الاستاتيكا - الصف الثالث الثانوي
الاستاتيكا
**************************************************
19
المراجعة النهائية - مادة الأحياء
الأحياء
**************************************************
23
مراجعة مادة الديناميكا - الصف الثالث الثانوي
الديناميكا
**************************************************
27
مراجعة ليلة الامتحان | علم النفس والاجتماع
علم النفس
**************************************************
29
مادة الديناميكا | الصف الثالث الثانوي | حصص مصر
الديناميكا
**************************************************
31
المراجعة النهائية | اللغة الإنجليزية | الصف الثالث الثانوي
اللغة الإنجليزية
**************************************************
34
المراجعة النهائية لمادة التاريخ | الصف الثالث الثانوي
التاريخ
**************************************************
36
ا

## 4- seperate the revision video from the orignal data <br />

In [10]:
## create new dataFrame with post_id and genere 
data = pd.DataFrame({"video_id": video_posts_id, "genere": video_posts_gen})

## select all video posts from orignal data
video_df = df[df["post_id"].isin(data["video_id"])]


In [11]:
## drop revision videos from orignal data 
## reset index and save data

clean_df = df.drop(video_df.index, axis=0)
clean_df.reset_index(drop=True, inplace=True)

clean_df.to_csv("hessas_nolive.csv", index=False)

In [12]:
## reset revision video index

video_df.reset_index(drop=True, inplace=True)

## generate genere columne and add data to it
genere = pd.Series(["" for i in range(len(video_df))])
video_df["genere"] = genere

for i in range(len(data)):
    video_df.loc[video_df[video_df.post_id == data["video_id"].loc[i]].index
                 , "genere"] = data["genere"].iloc[i]


C:\Users\208120~1\AppData\Local\Temp/ipykernel_1656/1321987086.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video_df["genere"] = genere
c:\users\20812018100700\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [13]:
video_df.sample(10)

,created_time,message,id,from,Translate,is_human,sentment,positive,neutral,negative,post_id,genere
24347,2021-07-08T17:01:22+0000,Zozo Ahmed \nShaza Mohamed \nMenna Galal \nNou...,339098197702579_339099667702432,NaN,Zozo Ahmed \nShaza Mohamed \nMenna Galal \nNou...,T,neutral,0.00,1.00,0.00,102899871666500_339098197702579,للغة العربية
1889,2021-07-26T12:13:50+0000,4,228127925851328_228137505850370,NaN,4,F,neutral,0.03,0.95,0.02,102899871666500_228127925851328,علم النفس
9104,2021-07-17T12:33:57+0000,Eman Sobhy,182917770520372_367608601597866,NaN,Eman Sobhy,T,neutral,0.00,1.00,0.00,102899871666500_182917770520372,الفلسفة
30652,2021-06-10T17:33:43+0000,ناقص,155073586669007_155097033333329,NaN,Incomplete,F,negative,0.00,0.00,1.00,102899871666500_155073586669007,الفلسفة
13552,2021-07-08T18:07:45+0000,التعبيرررررر,339098197702579_234066855029499,NaN,Expression,F,neutral,0.16,0.78,0.06,102899871666500_339098197702579,للغة العربية
34646,2021-03-03T18:39:24+0000,تربوي,425508425190847_425540235187666,NaN,educational,F,positive,0.56,0.39,0.05,102899871666500_425508425190847,علم النفس
35902,2021-02-22T17:33:39+0000,Gone,3794367117310529_3794407770639797,NaN,Gone,F,neutral,0.11,0.68,0.21,102899871666500_3794367117310529,اللغة الإنجليزية
6883,2021-07-23T13:27:28+0000,الفديو هيتحذف ولا لا,2935396270067847_301070405134318,NaN,the video will be deleted or not,F,neutral,0.01,0.64,0.35,102899871666500_2935396270067847,الفيزياء
1876,2021-07-26T12:17:48+0000,4,228127925851328_228141169183337,NaN,4,F,neutral,0.03,0.95,0.02,102899871666500_228127925851328,علم النفس
38118,2021-02-03T18:12:11+0000,غرب اوروبا,503157487333273_503197640662591,NaN,Western Europe,F,neutral,0.05,0.92,0.03,102899871666500_503157487333273,لدراسات الاجتماعية


In [14]:
video_df.to_csv(
    "hessas_live.csv", index=False)